# ETL Project

# 1. Dependencies

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base

import pymongo

In [2]:
# get chromedriver location
!which chromedriver

/usr/local/bin/chromedriver


# 2. EXTRACT

## 2.1 Visa Requirements for US Citizens -- web scraping `.html`

In [3]:
# launch chromedriver -- get an empty page
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
url = "https://en.wikipedia.org/wiki/Visa_requirements_for_United_States_citizens"
browser.visit(url)

In [5]:
tables = pd.read_html(browser.html)
visa_df = tables[0]
browser.quit()

In [6]:
# drop the unnecesary cols
col_to_drop = ['Allowed stay']
visa_df.drop(col_to_drop, axis = 1, inplace = True)

# rename cols
visa_df.rename(columns={'Country':'Country',
                        "Visa requirement":'visa_requirement',
                        "Notes (excluding departure fees)":'notes'},
              inplace = True)
visa_df.head()

,Country,visa_requirement,notes
0,Afghanistan,Visa required[2][3],Visitors born in Afghanistan do not require a ...
1,Albania,Visa not required[5][6],NaN
2,Algeria,Visa required[8][9],Persons may be denied entry if entering with a...
3,Andorra,Visa not required[10],NaN
4,Angola,eVisa[13][14][15],Visitors who have been granted an online pre-v...


In [7]:
# clean up the citation marks
for i in range(len(visa_df)):
    visa_df.iloc[i,1] = visa_df.iloc[i,1].split('[')[0]

In [8]:
visa_df['visa_requirement'].value_counts()

Visa not required                  107
Visa required                       30
Visa on arrival                     19
eVisa / Visa on arrival             18
eVisa                               11
Visitor's permit on arrival          1
Travel restricted                    1
eVisa / Tourist card on arrival      1
Electronic Travel Authority          1
Tourist Card required                1
Entry Permit on arrival              1
Visitor's Permit on arrival          1
Online Visa                          1
Name: visa_requirement, dtype: int64

In [9]:
auto_visa_key_words = ["on arrival", 'eVisa', 'Electronic', 'Online']
restrict_list = ["Tourist Card required", "Travel restricted"]

for i in range(len(visa_df)):
    for word in auto_visa_key_words:
        if word in visa_df['visa_requirement'][i]:
            visa_df['visa_requirement'][i] = "eVisa/Visa on arrival"
            continue
    if visa_df['visa_requirement'][i] in restrict_list:
        visa_df['visa_requirement'][i] = "Travel restricted"

In [10]:
visa_df['visa_requirement'].value_counts()

Visa not required        107
eVisa/Visa on arrival     54
Visa required             30
Travel restricted          2
Name: visa_requirement, dtype: int64

In [11]:
visa_df.set_index('Country',drop=True, inplace=True)
visa_df.head()

,visa_requirement,notes
Country,,
Afghanistan,Visa required,Visitors born in Afghanistan do not require a ...
Albania,Visa not required,NaN
Algeria,Visa required,Persons may be denied entry if entering with a...
Andorra,Visa not required,NaN
Angola,eVisa/Visa on arrival,Visitors who have been granted an online pre-v...


## 2.2 GDP by Country -- from `.csv` file

In [12]:
# The CSV file is in the resources directory
# The source website is https://data.worldbank.org/indicator/ny.gdp.mktp.cd
gdp_filename = "resources/GDP_by_country.csv"
gdp_df = pd.read_csv(gdp_filename)
gdp_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.498883e+09,2.390503e+09,2.549721e+09,2.534637e+09,2.581564e+09,2.649721e+09,2.691620e+09,2.646927e+09,2.700559e+09,NaN
1,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,537777811.1,548888895.6,546666677.8,751111191.1,800000044.4,1.006667e+09,...,1.243909e+10,1.585657e+10,1.780428e+10,2.000162e+10,2.056105e+10,2.048487e+10,1.990711e+10,1.936264e+10,2.019176e+10,1.936297e+10
2,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,7.030716e+10,8.379950e+10,1.117900e+11,1.280530e+11,1.367100e+11,1.457120e+11,1.161940e+11,1.011240e+11,1.221240e+11,1.057510e+11
3,Albania,ALB,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.204421e+10,1.192695e+10,1.289087e+10,1.231978e+10,1.277628e+10,1.322825e+10,1.138693e+10,1.186135e+10,1.302506e+10,1.505888e+10
4,Andorra,AND,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3.660531e+09,3.355695e+09,3.442063e+09,3.164615e+09,3.281585e+09,3.350736e+09,2.811489e+09,2.877312e+09,3.013387e+09,3.236544e+09


In [13]:
gdp_df.drop(columns=["Country Code","Indicator Name","Indicator Code"], inplace=True)
gdp_df.head()

,Country Name,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.498883e+09,2.390503e+09,2.549721e+09,2.534637e+09,2.581564e+09,2.649721e+09,2.691620e+09,2.646927e+09,2.700559e+09,NaN
1,Afghanistan,537777811.1,548888895.6,546666677.8,751111191.1,800000044.4,1.006667e+09,1.400000e+09,1.673333e+09,1.373333e+09,...,1.243909e+10,1.585657e+10,1.780428e+10,2.000162e+10,2.056105e+10,2.048487e+10,1.990711e+10,1.936264e+10,2.019176e+10,1.936297e+10
2,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.030716e+10,8.379950e+10,1.117900e+11,1.280530e+11,1.367100e+11,1.457120e+11,1.161940e+11,1.011240e+11,1.221240e+11,1.057510e+11
3,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.204421e+10,1.192695e+10,1.289087e+10,1.231978e+10,1.277628e+10,1.322825e+10,1.138693e+10,1.186135e+10,1.302506e+10,1.505888e+10
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.660531e+09,3.355695e+09,3.442063e+09,3.164615e+09,3.281585e+09,3.350736e+09,2.811489e+09,2.877312e+09,3.013387e+09,3.236544e+09


In [14]:
years = np.arange(1960,2018)
years_str = [str(yy) for yy in years]
gdp_df.drop(columns=years_str, inplace=True)
gdp_df.dropna(inplace=True)
gdp_df.rename(columns={"Country Name": "Country", "2018":"GDP 2018"}, inplace=True)
gdp_df.set_index("Country", inplace=True)
gdp_df.head()

,GDP 2018
Country,
Afghanistan,1.936297e+10
Angola,1.057510e+11
Albania,1.505888e+10
Andorra,3.236544e+09
Arab World,2.781330e+12


## 2.3 Top 100 Tourist Destinations Worldwide -- web scraping Brilliant Maps `.html` 

In [15]:
# URL of page to be scraped
url = "https://brilliantmaps.com/top-100-tourist-destinations/"

# Retrieve page
response = requests.get(url)

# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(response.text, 'html.parser')

# Examine the results, then determine element that contains sought info
# print(soup.prettify())

In [16]:
# Get the Top Destinations table, show the top 20 destinations
destinations = pd.read_html(response.text)
top_destinations=destinations[0]
top_destinations.head(5)

,Rank,City,Country,Tourists (Millions)
0,1,Hong Kong,Hong Kong,23.7
1,2,Singapore,Singapore,21.3
2,3,Bangkok,Thailand,15.8
3,4,London,UK,15.5
4,5,Macau,Macau,13.4


In [17]:
# Get a list of all popular cities aggregated by countries
test = top_destinations.copy().set_index('Rank')

city_list_df = pd.DataFrame({'count': test.groupby(['Country'])['City'].count(),
                            'city': test.groupby(['Country'])['City'].apply(list),
                            'tourists (mm)': test.groupby(['Country'])["Tourists (Millions)"].apply(list)})    
city_list_df.head()

,count,city,tourists (mm)
Country,,,
Argentina,1,[Buenos Aires],[3.2]
Australia,2,"[Sydney, Melbourne]","[2.7, 1.8]"
Austria,1,[Vienna],[4.3]
Azerbaijan,1,[Baku],[1.7]
Bahrain,1,[Manama],[2.0]


In [18]:
# Note that some countries have more than one city in the Global Top 100
# Find the most popular city to visit in each country
top_cities_by_country = top_destinations.loc[top_destinations.groupby(["Country"])["Rank"].idxmin()]
top_cities_by_country.rename(columns={"Country":"country", 
                                      "City":"city",
                                      "Tourists (Millions)":"tourists (mm)"}
                             , inplace=True)
top_cities_by_country.set_index("country", inplace=True)
top_cities_by_country.drop(columns="Rank", inplace=True)
top_cities_by_country.head(5)

,city,tourists (mm)
country,,
Argentina,Buenos Aires,3.2
Australia,Sydney,2.7
Austria,Vienna,4.3
Azerbaijan,Baku,1.7
Bahrain,Manama,2.0


## 2.4 Favorite dishes by country --  `.json` file 

In [19]:
# This json file comes from someone's github directory
# https://github.com/samayo/country-json/blob/master/src/country-by-national-dish.json

filepath = "resources/country-by-national-dish.json"
yummy_df = pd.read_json(filepath)
yummy_df.set_index("country", inplace=True)
yummy_df.head()

,dish
country,
Afghanistan,Kabuli Palaw
Albania,Tav� kosi
Algeria,Couscous
American Samoa,None
Andorra,None


## 2.5 World Countries and population -- web scraping `.html` 

In [23]:
# launch chromedriver -- get an empty page
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [24]:
url = "https://www.worldometers.info/geography/how-many-countries-are-there-in-the-world/"
browser.visit(url)

In [25]:
tables = pd.read_html(browser.html)
population_df = tables[0]

del population_df['#']
population_df.set_index('Country', inplace = True)
population_df.head()

,Population(2019),World Share,Land Area (Km²)
Country,,,
China,1420062022,18.4 %,9388211
India,1368737513,17.7 %,2973190
United States,329093110,4.3 %,9147420
Indonesia,269536482,3.5 %,1811570
Brazil,212392717,2.8 %,8358140


In [26]:
browser.quit()

# 3. TRANSFORM

**3.1 Clean-up country lists**

In [27]:
# function used to clean up the countries
def clean_up(df, unify_dict, drop_list):
    test = df.reset_index()
    for ct in drop_list:
        test = test[test['Country'] != ct]

    for i in range(len(test)):
        for key in unify_dict:
            if test.iloc[i, 0] in unify_dict[key]:
                test.iloc[i,0] = key
                
    return test

In [28]:
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

visa_ct_list = visa_df.index.to_list()
gdp_ct_list = gdp_df.index.to_list()
city_ct_list = city_list_df.index.to_list()
yummy_ct_list = yummy_df.index.to_list()
pop_ct_list = population_df.index.to_list()

lists = [visa_ct_list, gdp_ct_list, city_ct_list, yummy_ct_list, pop_ct_list]

for l in lists:
    print(f'\n{namestr(l, globals())[0]} : {len(l)}')
    print('-----------------------------')
    print(l[:10] +['...'])


visa_ct_list : 193
-----------------------------
['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', '...']

gdp_ct_list : 231
-----------------------------
['Afghanistan', 'Angola', 'Albania', 'Andorra', 'Arab World', 'United Arab Emirates', 'Argentina', 'Armenia', 'Antigua and Barbuda', 'Australia', '...']

city_ct_list : 53
-----------------------------
['Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Belgium', 'Brazil', 'Bulgaria', 'Cambodia', 'Canada', '...']

yummy_ct_list : 243
-----------------------------
['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', '...']

pop_ct_list : 195
-----------------------------
['China', 'India', 'United States', 'Indonesia', 'Brazil', 'Pakistan', 'Nigeria', 'Bangladesh', 'Russia', 'Mexico', '...']


**1) Visa Country list**

In [29]:
# countries in Population_df but not Visa_df
test_A = [x for x in pop_ct_list if x not in list(set(pop_ct_list) & set(visa_ct_list))]
print(sorted(test_A))

['Cabo Verde', 'Congo', 'Czechia', 'DR Congo', 'Holy See', 'Saint Kitts & Nevis', 'Sao Tome & Principe', 'St. Vincent & Grenadines', 'State of Palestine', 'Swaziland', 'United States']


In [30]:
# countries in Visa_df but not Population_df
test_B = [x for x in visa_ct_list if x not in list(set(pop_ct_list) & set(visa_ct_list))]
print(sorted(test_B))

['Cape Verde', 'Czech Republic', 'Democratic Republic of the Congo', 'Eswatini', 'Republic of the Congo', 'Saint Kitts and Nevis', 'Saint Vincent and the Grenadines', 'São Tomé and Príncipe', 'Vatican City']


In [31]:
# special: United States -- since Visa_df applies to US citizens
# Unify the two lists

unify_dict = {"Democratic Republic of the Congo" : ["Democratic Republic of the Congo", "DR Congo"],
 "Czech Republic": ["Czech Republic", "Czechia"],
 "Republic of the Congo" : ["Republic of the Congo", "Congo"],
 "Saint Kitts & Nevis": ["Saint Kitts and Nevis", "Saint Kitts & Nevis"],
 "St. Vincent & Grenadines" : ["St. Vincent & Grenadines","Saint Vincent and the Grenadines"],
 "Sao Tome & Principe": ["Sao Tome & Principe","São Tomé and Príncipe"],
 "Eswatini": ["Eswatini","Swaziland"],
 "Cape Verde": ["Cape Verde","Cabo Verde"]
}

drop_list = ["State of Palestine", "Holy See"]

In [32]:
visa_df = clean_up(visa_df, unify_dict, drop_list)
population_df = clean_up(population_df, unify_dict, drop_list)

In [33]:
df2 = pd.DataFrame([["Vatican City", 1000, '0 %', 0.44]], columns = list(population_df.columns))
population_df = population_df.append(df2, ignore_index=True)
population_df.head()

,Country,Population(2019),World Share,Land Area (Km²)
0,China,1420062022,18.4 %,9388211.0
1,India,1368737513,17.7 %,2973190.0
2,United States,329093110,4.3 %,9147420.0
3,Indonesia,269536482,3.5 %,1811570.0
4,Brazil,212392717,2.8 %,8358140.0


In [34]:
# update country name list 
population_df.set_index('Country', inplace = True)
visa_df.set_index('Country', inplace = True)

visa_ct_list = visa_df.index.to_list()
pop_ct_list = population_df.index.to_list()

lists = [visa_ct_list, gdp_ct_list, city_ct_list, yummy_ct_list, pop_ct_list]

**2) GDP Country list**

In [35]:
# countries in Population_df but not Visa_df
test_A = [x for x in pop_ct_list if x not in list(set(pop_ct_list) & set(visa_ct_list) & set(gdp_ct_list))]
print(sorted(test_A))

['Bahamas', 'Barbados', 'Brunei', 'Cape Verde', 'Cuba', "Côte d'Ivoire", 'Democratic Republic of the Congo', 'Egypt', 'Eritrea', 'Gambia', 'Iran', 'Kyrgyzstan', 'Laos', 'Liechtenstein', 'Micronesia', 'Monaco', 'North Korea', 'Republic of the Congo', 'Russia', 'Saint Kitts & Nevis', 'Saint Lucia', 'San Marino', 'Sao Tome & Principe', 'Slovakia', 'South Korea', 'South Sudan', 'St. Vincent & Grenadines', 'Syria', 'United States', 'Vatican City', 'Venezuela', 'Yemen']


In [36]:
# countries in Visa_df but not Population_df
test_B = [x for x in gdp_ct_list if x not in list(set(pop_ct_list) & set(visa_ct_list) & set(gdp_ct_list))]
sorted(test_B)

['Arab World',
 'Brunei Darussalam',
 'Cabo Verde',
 'Caribbean small states',
 'Central Europe and the Baltics',
 'Congo, Dem. Rep.',
 'Congo, Rep.',
 "Cote d'Ivoire",
 'Early-demographic dividend',
 'East Asia & Pacific',
 'East Asia & Pacific (IDA & IBRD countries)',
 'East Asia & Pacific (excluding high income)',
 'Egypt, Arab Rep.',
 'Euro area',
 'Europe & Central Asia',
 'Europe & Central Asia (IDA & IBRD countries)',
 'Europe & Central Asia (excluding high income)',
 'European Union',
 'Fragile and conflict affected situations',
 'Gambia, The',
 'Heavily indebted poor countries (HIPC)',
 'High income',
 'Hong Kong SAR, China',
 'IBRD only',
 'IDA & IBRD total',
 'IDA blend',
 'IDA only',
 'IDA total',
 'Korea, Rep.',
 'Kosovo',
 'Kyrgyz Republic',
 'Lao PDR',
 'Late-demographic dividend',
 'Latin America & Caribbean',
 'Latin America & Caribbean (excluding high income)',
 'Latin America & the Caribbean (IDA & IBRD countries)',
 'Least developed countries: UN classification',
 '

In [37]:
# special: United States -- since Visa_df applies to US citizens
# Unify the two lists

unify_dict = {"Democratic Republic of the Congo" : ["Democratic Republic of the Congo", 
                                                    "DR Congo", "Congo, Dem. Rep."],
 "Czech Republic": ["Czech Republic", "Czechia"],
 "Republic of the Congo" : ["Republic of the Congo", "Congo", "Congo, Rep."],
 "Saint Kitts & Nevis": ["Saint Kitts and Nevis", "Saint Kitts & Nevis", "St. Kitts and Nevis"],
 "St. Vincent & Grenadines" : ["St. Vincent & Grenadines",
                               "Saint Vincent and the Grenadines", 
                               "St. Vincent and the Grenadines"],
 "Sao Tome & Principe": ["Sao Tome & Principe","São Tomé and Príncipe", "Sao Tome and Principe"],
 "Eswatini": ["Eswatini","Swaziland"],
 "Côte d'Ivoire" : ["Côte d'Ivoire", "Cote d'Ivoire"],
 "Cape Verde": ["Cape Verde","Cabo Verde"],
 "Egypt" : ["Egypt, Arab Rep.","Egypt"],
 "Gambia" : ["Gambia, The", "Gambia"],
 "South Korea": ["Korea, Rep.","South Korea"],
 "Kyrgyzstan": ["Kyrgyz Republic", "Kyrgyzstan"],
 "Laos" : ["Lao PDR", "Laos"],
 "Micronesia" : ["Micronesia, Fed. Sts.", "Micronesia"],
 "Russia" : ["Russian Federation", "Russia"],
 "Slovakia" : ["Slovak Republic", "Slovakia"],
 "Saint Lucia" : ["St. Lucia", "Saint Lucia"],
 "Yemen" : ["Yemen, Rep.", "Yemen"],
 "Brunei" : ["Brunei Darussalam", "Brunei"]
}

drop_list = ["State of Palestine", "Holy See", 
             "Arab World","Caribbean small states","Central Europe and the Baltics","Early-demographic dividend",
             "East Asia & Pacific", "East Asia & Pacific (IDA & IBRD countries)",
             "East Asia & Pacific (excluding high income)","Euro area", "Europe & Central Asia",
             'Europe & Central Asia (IDA & IBRD countries)','Europe & Central Asia (excluding high income)',
             'European Union','Fragile and conflict affected situations',
             'Heavily indebted poor countries (HIPC)','High income','Hong Kong SAR, China','IBRD only',
             'IDA & IBRD total','IDA blend','IDA only', 'IDA total','Kosovo',
             'Late-demographic dividend', 'Latin America & Caribbean',
             'Latin America & Caribbean (excluding high income)',
             'Latin America & the Caribbean (IDA & IBRD countries)','Least developed countries: UN classification',
             'Low & middle income','Low income','Lower middle income','Macao SAR, China',
             'Middle East & North Africa','Middle income', 'North America','OECD members','Other small states',
             'Pacific island small states','Post-demographic dividend','Pre-demographic dividend','Puerto Rico',
             'Small states','South Asia', 'South Asia (IDA & IBRD)','Sub-Saharan Africa',
             'Sub-Saharan Africa (IDA & IBRD countries)','Sub-Saharan Africa (excluding high income)',
             'Turks and Caicos Islands','West Bank and Gaza', 'World', 'Upper middle income'
            ]

In [38]:
gdp_df = clean_up(gdp_df, unify_dict, drop_list)
population_df = clean_up(population_df, unify_dict, drop_list)

In [39]:
# update country name list 
population_df.set_index('Country', inplace = True)
gdp_df.set_index('Country', inplace = True)

gdp_ct_list = gdp_df.index.to_list()
pop_ct_list = population_df.index.to_list()
lists = [visa_ct_list, gdp_ct_list, city_ct_list, yummy_ct_list, pop_ct_list]

**3) yummy Country list**

In [40]:
# countries in Population_df but not Visa_df
test_A = [x for x in pop_ct_list if x not in list(set(pop_ct_list) & 
                                                  set(visa_ct_list) & 
                                                  set(gdp_ct_list) &
                                                  set(yummy_ct_list))]
print(sorted(test_A))

['Bahamas', 'Barbados', 'Cuba', "Côte d'Ivoire", 'Democratic Republic of the Congo', 'Eritrea', 'Eswatini', 'Fiji', 'Iran', 'Libya', 'Liechtenstein', 'Micronesia', 'Monaco', 'Montenegro', 'North Korea', 'Republic of the Congo', 'Russia', 'Saint Kitts & Nevis', 'San Marino', 'Sao Tome & Principe', 'Serbia', 'South Sudan', 'Sri Lanka', 'St. Vincent & Grenadines', 'Syria', 'Timor-Leste', 'United States', 'Vatican City', 'Venezuela']


In [41]:
# countries in Visa_df but not Population_df
test_A = [x for x in yummy_ct_list if x not in list(set(pop_ct_list) & 
                                                  set(visa_ct_list) & 
                                                  set(gdp_ct_list) &
                                                  set(yummy_ct_list))]
sorted(test_B)

['Arab World',
 'Brunei Darussalam',
 'Cabo Verde',
 'Caribbean small states',
 'Central Europe and the Baltics',
 'Congo, Dem. Rep.',
 'Congo, Rep.',
 "Cote d'Ivoire",
 'Early-demographic dividend',
 'East Asia & Pacific',
 'East Asia & Pacific (IDA & IBRD countries)',
 'East Asia & Pacific (excluding high income)',
 'Egypt, Arab Rep.',
 'Euro area',
 'Europe & Central Asia',
 'Europe & Central Asia (IDA & IBRD countries)',
 'Europe & Central Asia (excluding high income)',
 'European Union',
 'Fragile and conflict affected situations',
 'Gambia, The',
 'Heavily indebted poor countries (HIPC)',
 'High income',
 'Hong Kong SAR, China',
 'IBRD only',
 'IDA & IBRD total',
 'IDA blend',
 'IDA only',
 'IDA total',
 'Korea, Rep.',
 'Kosovo',
 'Kyrgyz Republic',
 'Lao PDR',
 'Late-demographic dividend',
 'Latin America & Caribbean',
 'Latin America & Caribbean (excluding high income)',
 'Latin America & the Caribbean (IDA & IBRD countries)',
 'Least developed countries: UN classification',
 '

**4) 100 Travel desitination Country list**

In [42]:
# countries in Population_df but not Visa_df
test_A = [x for x in pop_ct_list if x not in list(set(pop_ct_list) & 
                                                  set(visa_ct_list) & 
                                                  set(gdp_ct_list) &
                                                  set(yummy_ct_list) & set(city_ct_list))]
print(sorted(test_A))

['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Armenia', 'Bahamas', 'Bangladesh', 'Barbados', 'Belarus', 'Belize', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brunei', 'Burkina Faso', 'Burundi', 'Cameroon', 'Cape Verde', 'Central African Republic', 'Chad', 'Chile', 'Colombia', 'Comoros', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', "Côte d'Ivoire", 'Democratic Republic of the Congo', 'Denmark', 'Djibouti', 'Dominica', 'Ecuador', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'Gabon', 'Gambia', 'Georgia', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Iceland', 'Iran', 'Iraq', 'Israel', 'Jamaica', 'Kazakhstan', 'Kiribati', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Madagascar', 'Malawi', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', '

In [43]:
# countries in Visa_df but not Population_df
test_A = [x for x in city_ct_list if x not in list(set(pop_ct_list) & 
                                                  set(visa_ct_list) & 
                                                  set(gdp_ct_list) &
                                                  set(yummy_ct_list) & set(city_ct_list))]
print(sorted(test_B))

['Arab World', 'Brunei Darussalam', 'Cabo Verde', 'Caribbean small states', 'Central Europe and the Baltics', 'Congo, Dem. Rep.', 'Congo, Rep.', "Cote d'Ivoire", 'Early-demographic dividend', 'East Asia & Pacific', 'East Asia & Pacific (IDA & IBRD countries)', 'East Asia & Pacific (excluding high income)', 'Egypt, Arab Rep.', 'Euro area', 'Europe & Central Asia', 'Europe & Central Asia (IDA & IBRD countries)', 'Europe & Central Asia (excluding high income)', 'European Union', 'Fragile and conflict affected situations', 'Gambia, The', 'Heavily indebted poor countries (HIPC)', 'High income', 'Hong Kong SAR, China', 'IBRD only', 'IDA & IBRD total', 'IDA blend', 'IDA only', 'IDA total', 'Korea, Rep.', 'Kosovo', 'Kyrgyz Republic', 'Lao PDR', 'Late-demographic dividend', 'Latin America & Caribbean', 'Latin America & Caribbean (excluding high income)', 'Latin America & the Caribbean (IDA & IBRD countries)', 'Least developed countries: UN classification', 'Low & middle income', 'Low income', '

# 4. LOAD

In [44]:
visa_df.to_csv("Output/visa_requirement_us_citizen.csv")
visa_df.head()

,visa_requirement,notes
Country,,
Afghanistan,Visa required,Visitors born in Afghanistan do not require a ...
Albania,Visa not required,NaN
Algeria,Visa required,Persons may be denied entry if entering with a...
Andorra,Visa not required,NaN
Angola,eVisa/Visa on arrival,Visitors who have been granted an online pre-v...


In [45]:
gdp_df.rename(columns={"GDP 2018": "gdp_2018"}, inplace=True)
gdp_df.to_csv("Output/GDP_2018.csv")
gdp_df.head()

,gdp_2018
Country,
Afghanistan,1.936297e+10
Angola,1.057510e+11
Albania,1.505888e+10
Andorra,3.236544e+09
United Arab Emirates,4.141790e+11


In [46]:
city_list_df.rename(columns={"tourists (mm)": "tourists_mm",
                             "count": "city_count",
                             "city": "cities"
                            }, inplace=True)
city_list_df.to_csv("Output/top_100_desination.csv")
city_list_df.head()

,city_count,cities,tourists_mm
Country,,,
Argentina,1,[Buenos Aires],[3.2]
Australia,2,"[Sydney, Melbourne]","[2.7, 1.8]"
Austria,1,[Vienna],[4.3]
Azerbaijan,1,[Baku],[1.7]
Bahrain,1,[Manama],[2.0]


In [47]:
yummy_df.to_csv("Output/yummy.csv")
yummy_df.head()

,dish
country,
Afghanistan,Kabuli Palaw
Albania,Tav� kosi
Algeria,Couscous
American Samoa,None
Andorra,None


In [48]:
population_df.rename(columns={"Population(2019)": "population",
                       "World Share":"world_share",
                       "Land Area (Km²)": "land_area"
                      }, inplace=True)
population_df.to_csv("Output/world_population.csv")
population_df.head()

,population,world_share,land_area
Country,,,
China,1420062022,18.4 %,9388211.0
India,1368737513,17.7 %,2973190.0
United States,329093110,4.3 %,9147420.0
Indonesia,269536482,3.5 %,1811570.0
Brazil,212392717,2.8 %,8358140.0


#### 4.1 SQLAlchemy DataBase

In [72]:
# create table 
from sqlalchemy import Column, Integer, String, Float, BigInteger
from sqlalchemy.orm import Session

In [73]:
database_path = "resources/country_db"
engine = create_engine(f"sqlite:///{database_path}")

In [74]:
conn = engine.connect()
Base = declarative_base()
session = Session(bind = engine)

In [75]:
class Visa(Base):
    #Tell SQLAlchemy what the table name is and if there's any table-specific arguments it should know about
    __tablename__ = 'Visa'
    __table_args__ = {'sqlite_autoincrement': True}
    #tell SQLAlchemy the name of column and its attributes:
    id = Column(Integer, primary_key=True) 
    country = Column(String)
    visa_requirement = Column(String)
    notes = Column(String)

In [76]:
class GDP(Base):
    #Tell SQLAlchemy what the table name is and if there's any table-specific arguments it should know about
    __tablename__ = 'GDP'
    __table_args__ = {'sqlite_autoincrement': True}
    #tell SQLAlchemy the name of column and its attributes:
    id = Column(Integer, primary_key=True) 
    country = Column(String)
    gdp_2018 = Column(BigInteger)

In [77]:
class Population(Base):
    #Tell SQLAlchemy what the table name is and if there's any table-specific arguments it should know about
    __tablename__ = 'Population'
    __table_args__ = {'sqlite_autoincrement': True}
    #tell SQLAlchemy the name of column and its attributes:
    id = Column(Integer) 
    country = Column(String, primary_key=True)
    population = Column(BigInteger)
    world_share  = Column(String)
    land_area = Column(Integer)

In [78]:
class Yummy(Base):
    #Tell SQLAlchemy what the table name is and if there's any table-specific arguments it should know about
    __tablename__ = 'Yummy'
    __table_args__ = {'sqlite_autoincrement': True}
    #tell SQLAlchemy the name of column and its attributes:
    id = Column(Integer) 
    country = Column(String, primary_key=True)
    dish = Column(String)

In [79]:
class City_list(Base):
    #Tell SQLAlchemy what the table name is and if there's any table-specific arguments it should know about
    __tablename__ = 'City_list'
    __table_args__ = {'sqlite_autoincrement': True}
    #tell SQLAlchemy the name of column and its attributes:
    id = Column(Integer) 
    country = Column(String, primary_key=True)
    city_count = Column(String)
    cities = Column(String)
    tourists_mm = Column(String)

In [80]:
visa = pd.read_csv("Output/visa_requirement_us_citizen.csv")
visa.to_sql(con=engine, index_label='id', name=Visa.__tablename__, if_exists='replace')

In [81]:
gdp = pd.read_csv("Output/GDP_2018.csv")
gdp.to_sql(con=engine, index_label='id', name=GDP.__tablename__, if_exists='replace')

In [82]:
population = pd.read_csv("Output/world_population.csv")
population.to_sql(con=engine, index_label='id', name=Population.__tablename__, if_exists='replace')

In [83]:
yummy = pd.read_csv("Output/yummy.csv")
yummy.to_sql(con=engine, index_label='id', name=Yummy.__tablename__, if_exists='replace')

In [84]:
city_list = pd.read_csv("Output/top_100_desination.csv")
city_list.to_sql(con=engine, index_label='id', name=City_list.__tablename__, if_exists='replace')

# 5. QUERY TIME!

In [70]:
rows = session.query(Visa.country, Visa.visa_requirement, GDP.gdp_2018).filter(Visa.country == GDP.country)

In [71]:
for record in rows:
    print(record)

('Afghanistan', 'Visa required', 19362969582.0)
('Albania', 'Visa not required', 15058879129.0)
('Algeria', 'Visa required', 180689000000.0)
('Andorra', 'Visa not required', 3236543909.0)
('Angola', 'eVisa/Visa on arrival', 105751000000.0)
('Antigua and Barbuda', 'Visa not required', 1623804016.0)
('Argentina', 'Visa not required', 518475000000.0)
('Armenia', 'Visa not required', 12433089919.0)
('Australia', 'eVisa/Visa on arrival', 1432200000000.0)
('Austria', 'Visa not required', 455737000000.0)
('Azerbaijan', 'eVisa/Visa on arrival', 46939529412.0)
('Bahrain', 'eVisa/Visa on arrival', 37746196809.0)
('Bangladesh', 'eVisa/Visa on arrival', 274025000000.0)
('Belarus', 'Visa not required', 59662495092.0)
('Belgium', 'Visa not required', 531767000000.0)
('Belize', 'Visa not required', 1925000000.0)
('Benin', 'eVisa/Visa on arrival', 10358985583.0)
('Bhutan', 'Visa required', 2534965163.0)
('Bolivia', 'eVisa/Visa on arrival', 40287647757.0)
('Bosnia and Herzegovina', 'Visa not required',

## 5.1 In your next vacation...
1. country w/o visa requiremnt 
2. list of favourate cities
3. favourite dish in that country

In [91]:
sel = [Visa.country, City_list.cities, Yummy.dish, Population.population, GDP.gdp_2018,]
rows = session.query(*sel).filter(Visa.visa_requirement == "Visa not required",
                                  Visa.country == Population.country,
                                  Visa.country == City_list.country, 
                                  Visa.country == Yummy.country,
                                  Visa.country == GDP.country
                                 )

In [93]:
i = 1
for record in rows:
    print(i,". ",  record)
    i += 1

1 .  ('Argentina', "['Buenos Aires']", 'Asado', 45101781, 518475000000.0)
2 .  ('Austria', "['Vienna']", 'Wiener schnitzel, Tafelspitz', 8766201, 455737000000.0)
3 .  ('Belgium', "['Brussels']", 'Moules-frites', 11562784, 531767000000.0)
4 .  ('Brazil', "['Rio de Janeiro', 'Sao Paulo']", 'Feijoada', 212392717, 1868630000000.0)
5 .  ('Bulgaria', "['Sofia', 'Burgas']", 'Banitsa, Bob chorba', 6988739, 65132951116.0)
6 .  ('Canada', "['Toronto', 'Vancouver']", 'Kraft Dinner', 37279811, 1709330000000.0)
7 .  ('Czech Republic', "['Prague']", 'roast pork with dumplings and sauerkraut', 10630589, 244105000000.0)
8 .  ('Dominican Republic', "['Punta Cana']", 'rice, red beans, and meat', 10996774, 81298585403.0)
9 .  ('France', "['Paris', 'Nice']", 'Cr�pe, Pot-au-feu,Macaron, Bisque', 65480710, 2777540000000.0)
10 .  ('Germany', "['Berlin', 'Munich', 'Frankfurt']", 'Currywurst, Sauerbraten, Bratwurst', 82438639, 3996760000000.0)
11 .  ('Hungary', "['Budapest']", 'Guly�s,', 9655361, 155703000000.